In [1]:
import pandas as pd
import yfinance as yf
import datetime as dt
from pandas_datareader import data as pdr
import os
import numpy as np
import schedule
import time
from datetime import date
import yagmail
import keyring


yf.pdr_override()

In [2]:
# List of tickers to iterate through. 

dow_list = ["AXP", "AMGN", 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', "GS", "HD", "HON", "IBM", "INTC", "JNJ", "KO", "JPM", "MCD",
           "MMM", "MRK", "MSFT", "NKE", "PG", "TRV", "UNH", "CRM", "VZ", "V", "WBA", "WMT", "DIS", "DOW"]


ev_list = ["AYRO", "NRG", "IDRV", "DRIV", "JCI", "RIDE", "SOLO", "LI", "VWAGY", "NKLA", "NIO", "TSLA", "TM", 'WKHS']


semiconductor_list = ["TSM", "ASML", "QCOM", "AMD", "NXPI", "SWKS", "TXN", "MRAAY", "NXPI", "MU", "IFNNY"] 


biotech_list = ["BDSI", "ARDX", "APTO", "AGLE", "BCRX", "ASMB", "DVAX", "WVE", "KDMN", "ISEE", "SRPT", "ALNY", "INCY",
                "ILMN", "REGN", "BMRN", "AMGN", "VRTX", "ALXN", "BIIB", "GILD", "HZNP"]

gaming_list = ["PENN", "DKNG", "CZR", "BETZ", "MGM"]


flight_list = ["UAL", "DAL", "AAL", "SAVE", "LUV", "JBLU", "ALK"]


fintech_list = ["SQ", "PYPL", "GS", "GDOT", "MELI"]


blockchain_list = ["NVDA", "CME", "IBM", "MA", "DOCU", "AMZN", "BLOK", "MARA", "MSTR", "BLCN", "CAN",
                  "V"]

# Random stocks to test
random_list = ["FUNC", "SPPI", "FXO", "MGPI", "GWRS", "NUV", "FLGE", "NCMI", "ODP", "CHKEL", "OZK", "FRHC"]

energy_list = ["EOG", "LNG", "BP", "EPD", "SU", "CVX", "RUN", "ENPH", "DVN"]

infrastructure_list = ['FAST', "MLM", "NSC", "UNP"]

potentially_undervalued = ["WFC", "XOM", "CVX", "EOG", "MPC", "RBBN", "FTNT", "NET"]

total_list = set(dow_list + ev_list + semiconductor_list + biotech_list + flight_list + fintech_list + blockchain_list +
             energy_list + infrastructure_list)

unique_list = set(total_list)


In [3]:
def stock_data(stock_list):
    # Creates an empty dictionary, then grabs the data from yahoo finance

    start_date = dt.datetime.now() - dt.timedelta(days=90)
    end_date = dt.datetime.now()
    ticker_dictionary = {}

    # Specify chosen list of stocks below
    for i in stock_list:
        tickerSymbol = i
        tickerData = yf.Ticker(tickerSymbol)
        tickerDF = tickerData.history(period = '1d', start=start_date, end=end_date)
        ticker_dictionary[i] = tickerDF
    
    return ticker_dictionary

In [4]:
def stock_picks(trigger):
    percent_change = []
    purchase = 0
    Buy = []
    Sell = []
    for i in range(0, len(trigger)):
        Close = trigger["Close"][i]
        Close = round(Close, 2)
        
        if trigger['MACD'][i] > trigger['Trigger'][i]:
            Sell.append(np.nan)
            
            if purchase != 1:
                Buy.append(trigger['Close'][i])
                purchase = 1
                bp = Close
                print("Buying @ " + str(round(Close, 2)) + " on " + str(trigger.index[i]))

            else:
                Buy.append(np.nan)

        elif trigger['MACD'][i] < trigger['Trigger'][i]:
            Buy.append(np.nan)
            
            if purchase != 0:
                Sell.append(trigger['Close'][i])
                purchase = 0
                sp = Close
                pc = ((sp - bp)/bp * 100)
                pc = round(pc, 2)
                percent_change.append(pc)
                print("Sell now @ " +str(round(Close, 2)) + " on " + str(trigger.index[i]))
                
            else:
                Sell.append(np.nan)

        else:
            Sell.append(np.nan)
            Buy.append(np.nan)
         

            
    
    print(percent_change)
    print("     TOTAL GAINS/LOSSES: " + str(sum(percent_change)) + "%")
    print("")

    
    return(Buy, Sell)

In [5]:
# calculate short/long term EMA, MACD, and signal line indicators

def signal_function(ticker_dictionary):

    for i in ticker_dictionary:

        df = ticker_dictionary[i]

    # the short and long EMAs can easily be changed by modifying the span.

    # short ema 

        short = df.Close.ewm(span=12, adjust=False).mean()

    # long ema

        long = df.Close.ewm(span=26, adjust=False).mean()

    # MACD line

        MACD = short - long

    # signal line

        Trigger_line = MACD.ewm(span=9, adjust = False).mean()

    # Create new columns for data
        df["MACD"] = MACD
        df['Trigger'] = Trigger_line

    

        print("****CHECKING STOCK " + str(i) + "****")
        df2 = df
        test = stock_picks(df2)

        df['Buy_Trigger'] = test[0]
        df['Sell_Trigger'] = test[1]
        
        
    return(ticker_dictionary)

In [6]:
# Daily Buy and Sell lists
def daily_list(ticker_dictionary):
    yesterday = []
    sell_today = []
    buy_today = []

    
    for i in ticker_dictionary:
        new_df = ticker_dictionary[i]
        new_df["Stock"] = str(i)
        last_rows = new_df[["Buy_Trigger", "Sell_Trigger", "Stock"]].tail(1) 
        last_rows = last_rows.dropna(axis='rows', thresh=2)

        buy_stock = last_rows.dropna(subset = ["Buy_Trigger"])
        buy_stock = buy_stock[["Stock", "Buy_Trigger"]]
        buy_stock = buy_stock.rename_axis("", axis = 0)
        buy_stock.columns = [''] * len(buy_stock.columns)

        if not buy_stock.empty:
            buy_today.append(buy_stock)

        sell_stock = last_rows.dropna(subset = ["Sell_Trigger"])
        sell_stock = sell_stock[["Stock", "Sell_Trigger"]]
        sell_stock = sell_stock.rename_axis("", axis = 0)
        sell_stock.columns = [''] * len(sell_stock.columns)

        if not sell_stock.empty:
            sell_today.append(sell_stock)

        if not last_rows.empty:
            yesterday.append(last_rows)

    print("Buy list: " + str(buy_today))
    print("Sell list: " + str(sell_today))
    
    return(buy_today, sell_today)

In [7]:
def email(buy, sell):
    sender_email = os.environ.get("SENDER_EMAIL")
    receiver_email = os.environ.get('RECEIVER_EMAIL')
    sender_password = os.environ.get("SENDER_PASSWORD")
    subject = "Daily Stock Information"

    yag = yagmail.SMTP(user=sender_email, password=sender_password)

    contents = [
      ' "I think you will find',
        "\n",
       'When your death takes its toll',
        "\n",
        'All the money you made',
        "\n",
        'Will never buy back your soul" - Bob Dylan',
        "\n",
        "\n",
        "Here is Today's Buy List: ", 
        "\n",
         str(buy),
        "\n",
        "Here is Today's Sell List: ",
        "\n", 
         str(sell),
        ]

    yag.send(receiver_email, subject, contents)

In [8]:
def main():
    Buy = []
    Sell = []
    percent_change = []
    yesterday = []
    sell_today = []
    buy_today = []

    data = stock_data(energy_list)
    f = signal_function(data)
    x, y = daily_list(f)
    email(x, y)

In [9]:
if __name__ == "__main__":
    main()

****CHECKING STOCK EOG****
Buying @ 62.05 on 2021-01-12 00:00:00
Sell now @ 55.03 on 2021-01-21 00:00:00
Buying @ 56.61 on 2021-02-05 00:00:00
Sell now @ 73.19 on 2021-03-17 00:00:00
[-11.31, 29.29]
     TOTAL GAINS/LOSSES: 17.979999999999997%

****CHECKING STOCK LNG****
Buying @ 67.02 on 2021-01-20 00:00:00
Sell now @ 66.0 on 2021-01-22 00:00:00
Buying @ 64.9 on 2021-02-04 00:00:00
Sell now @ 71.26 on 2021-03-19 00:00:00
[-1.52, 9.8]
     TOTAL GAINS/LOSSES: 8.280000000000001%

****CHECKING STOCK BP****
Buying @ 24.68 on 2021-01-12 00:00:00
Sell now @ 23.7 on 2021-01-21 00:00:00
Buying @ 22.87 on 2021-02-16 00:00:00
Sell now @ 25.41 on 2021-03-18 00:00:00
[-3.97, 11.11]
     TOTAL GAINS/LOSSES: 7.139999999999999%

****CHECKING STOCK EPD****
Buying @ 21.92 on 2021-01-12 00:00:00
Sell now @ 21.09 on 2021-01-22 00:00:00
Buying @ 21.22 on 2021-02-09 00:00:00
Sell now @ 22.29 on 2021-03-18 00:00:00
Buying @ 22.92 on 2021-04-09 00:00:00
[-3.79, 5.04]
     TOTAL GAINS/LOSSES: 1.25%

****CHEC